## Set up google colab environment

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [2]:
from sample_extraction_V2_utils import *
import scipy.io as sio

## Load Indian Pines Dataset

In [3]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [4]:
data = uIndianPines['indian_pines_corrected']
ground_truth = gt_IndianPines['indian_pines_gt']

In [5]:
data.shape

(145, 145, 200)

In [6]:
ground_truth.shape

(145, 145)

# Dimensionality reduction of bands/channels

In [ ]:
nb = 200

# solve the linear equations
x + y = 64

(nb/64) (x)  x + (nb/64 + 1)(y) = nb

In [ ]:
first_interval = np.floor(nb/64)
second_interval = np.floor(nb/64) + 1

In [ ]:
first_interval, second_interval

(3.0, 4.0)

In [ ]:
  a = np.array([[1,1], [first_interval,second_interval]])
  b = np.array([64,nb])
  x = np.linalg.solve(a, b)
  x

array([56.,  8.])

## Distrubution of samples for each class

In [7]:
class_distribution = pd.DataFrame(np.unique(ground_truth, return_counts = True))
class_distribution = class_distribution.transpose()
class_distribution.columns = ['class','samples']
class_distribution

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [8]:
classes , counts = np.unique(ground_truth, return_counts = True)
classes = classes[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with overlap ratio 1 and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [9]:
pretrain_results = pretrain_source_models(percentages = [60,70,80,90],
                                          classes = classes,
                                          cube_size = 20,
                                          overlap_ratios = [0.8,0.9,1],
                                          data = data,
                                          ground_truth = ground_truth,
                                          batch_size = 20,
                                          channels = 64,
                                          epochs = 100,
                                          Verbosity = 1,
                                          accuracies = [],
                                          learning_rate = 0.0001,
                                          source_dataset = 'indian_pines')

Streaming output truncated to the last 5000 lines.

Epoch 00040: val_categorical_accuracy did not improve from 0.71083
Epoch 41/100
118/118 [==============================] - 2s 15ms/step - loss: 2.2709 - categorical_accuracy: 0.9375 - val_loss: 6.0971 - val_categorical_accuracy: 0.3879

Epoch 00041: val_categorical_accuracy did not improve from 0.71083
Epoch 42/100
118/118 [==============================] - 2s 15ms/step - loss: 2.2754 - categorical_accuracy: 0.9406 - val_loss: 3.1765 - val_categorical_accuracy: 0.6541

Epoch 00042: val_categorical_accuracy did not improve from 0.71083
Epoch 43/100
118/118 [==============================] - 2s 15ms/step - loss: 2.2480 - categorical_accuracy: 0.9359 - val_loss: 3.2950 - val_categorical_accuracy: 0.6484

Epoch 00043: val_categorical_accuracy did not improve from 0.71083
Epoch 44/100
118/118 [==============================] - 2s 14ms/step - loss: 2.2455 - categorical_accuracy: 0.9294 - val_loss: 3.0495 - val_categorical_accuracy: 0.6344



In [10]:
pretrain_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,477,323,60,67.18
1,0.8,555,245,70,69.80
2,0.8,636,164,80,75.00
3,0.8,715,85,90,69.41
4,0.9,2348,1570,60,72.55
5,0.9,2740,1178,70,71.56
6,0.9,3132,786,80,82.32
7,0.9,3523,395,90,82.53
8,1.0,4740,3166,60,74.61
9,1.0,5530,2376,70,73.91


,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,397,403,50,68.24
1,0.8,477,323,60,69.35
2,0.8,555,245,70,70.20
3,0.8,636,164,80,77.44
4,0.9,1956,1962,50,74.97
5,0.9,2348,1570,60,70.64
6,0.9,2740,1178,70,75.38
7,0.9,3132,786,80,74.43
8,1.0,3951,3955,50,73.10
9,1.0,4740,3166,60,69.14
